In [2]:
import os
import jsonlines

In [ ]:
## Store seperate text

datasets = ["litbank", "fantasy"]
splits = ["dev","test"]
for dataset in datasets:
    for split in splits:
        if dataset == "litbank":
            input_jsonlines = f"../mei-data/raw_data/{dataset}/longformer_speaker/0/{split}.4096.mei.jsonlines"
        else:
            input_jsonlines = f"../mei-data/raw_data/{dataset}/longformer_speaker/{split}.4096.mei.jsonlines"
        output_dir = f"../mei-data/raw_data/{dataset}/text/{split}"
        os.makedirs(output_dir, exist_ok=True)
        if split == "dev" and dataset == "fantasy":
            continue
        with jsonlines.open(input_jsonlines) as reader:
            for obj in reader:
                doc_key = obj["doc_key"]
                text = " ".join(obj["orig_tokens"])
                with open(f"{output_dir}/{doc_key}.txt", "w") as f:
                    f.write(text)

In [3]:
def generate_html(words, clusters):
    html_parts = words[:]
    cluster_map = {}

    # Create a mapping from word index to its clusters
    for cluster_num, cluster in enumerate(clusters, start=1):
        for start, end in cluster:
            for idx in range(start, end + 1):
                if idx in cluster_map:
                    cluster_map[idx].append(cluster_num)
                else:
                    cluster_map[idx] = [cluster_num]

    # Build the HTML string
    html_string = ""
    for idx, word in enumerate(words):
        if idx in cluster_map:
            clusters_str = ','.join(map(str, cluster_map[idx]))
            html_string += f"<span style='color:blue'>[{word}]<sub>{clusters_str}</sub></span> "
        else:
            html_string += word + " "

    return html_string.strip()

# Example usage
words = ["word1", "word2", "word3", "word4", "word5", "word6","word7","word8"]
clusters = [
    [[0, 2], [4, 5]],
    [[6, 7]]
]

html_output = generate_html(words, clusters)
print(html_output)

<span style='color:blue'>[word1]<sub>1</sub></span> <span style='color:blue'>[word2]<sub>1</sub></span> <span style='color:blue'>[word3]<sub>1</sub></span> word4 <span style='color:blue'>[word5]<sub>1</sub></span> <span style='color:blue'>[word6]<sub>1</sub></span> <span style='color:blue'>[word7]<sub>2</sub></span> <span style='color:blue'>[word8]<sub>2</sub></span>


In [18]:
def highlight_clusters(text_words, golden_clusters):
    # Create a list to store the formatted words
    formatted_words = list(text_words)
    
    # Sort the mentions based on their start index to handle nested mentions properly
    mention_to_cluster = {}
    all_mentions_pos = []
    for cluster_ind,cluster in enumerate(golden_clusters):
        for mention in cluster:
            mention = tuple(mention)
            mention_to_cluster[mention] = cluster
            all_mentions_pos.append((mention[0],"<span style='color:blue;'>["))
            all_mentions_pos.append((mention[1],f"]<sub>{cluster_ind}</sub></span>"))
    all_mentions_pos.sort()

    for pos,element in all_mentions_pos:
       
        if element[-1] == "[":
            formatted_words[pos] = element + formatted_words[pos]
        else:
            formatted_words[pos] = formatted_words[pos] + element
    
    return " ".join(formatted_words)
        
# Example usage
text_words = ["This", "is", "a", "test", "text", "with", "coreference", "clusters", "."]
golden_clusters = [[[1, 3], [4, 5]], [[2, 2]]]
highlighted_text = highlight_clusters(text_words, golden_clusters)
print(highlighted_text)

This <span style='color:blue;'>[is <span style='color:blue;'>[a]<sub>1</sub></span> test]<sub>0</sub></span> <span style='color:blue;'>[text with]<sub>0</sub></span> coreference clusters .


In [12]:
a = ["1","2","3","4","5",]
a.insert(0,"6")
a

['6', '1', '2', '3', '4', '5']